This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

# Spherical Linear Interpolation (Slerp)

"Great arc in-betweening"

The term "Slerp" for "**s**pherical **l**inear int**erp**olation"
has been coined by
<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>
(section 3.3).
It is defined as:

\begin{equation*}
\operatorname{Slerp}(q_1, q_2; u) =
q_1 \left(q_1^{-1} q_2\right)^u
\end{equation*}

The parameter $u$ moves from $0$ (where the expression simplifies to $q_1$)
to $1$ (where the expression simplifies to $q_2$).

The [Wikipedia article for Slerp](https://en.wikipedia.org/wiki/Slerp#Quaternion_Slerp)
provides four equivalent ways to describe the same thing:

\begin{align*}
\operatorname{Slerp}(q_0, q_1; t)
& = q_0 \left(q_0^{-1} q_1\right)^t \\
& = q_1 \left(q_1^{-1} q_0\right)^{1-t} \\
& = \left(q_0 q_1^{-1}\right)^{1-t} q_1 \\
& = \left(q_1 q_0^{-1}\right)^t q_0
\end{align*}

Shoemake also provides an alternative formulation:

\begin{equation*}
\operatorname{Slerp}(q_1, q_2; u) =
\frac{\sin (1-u) \theta}{\sin \theta} q_1 +
\frac{\sin u \theta}{\sin \theta} q_2,
\end{equation*}

where the dot product
$q_1 \cdot q_2 = \cos \theta$.

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

In [ ]:
def slerp(one, two, t):
    return (two * one.inverse())**t * one

In [ ]:
q1 = angles2quat(45, -20, -60)
q2 = angles2quat(-45, 20, 30)

In [ ]:
times = np.linspace(0, 1, 50)

In [ ]:
ani = animate_rotations({
    'slerp(q1, q2)': slerp(q1, q2, times),
    'slerp(q1, -q2)': slerp(q1, -q2, times),
}, figsize=(6, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

## Piecewise Slerp

In [ ]:
from splines.quaternion import PiecewiseSlerp

In [ ]:
s = PiecewiseSlerp([
    angles2quat(0, 0, 0),
    angles2quat(90, 0, 0),
    angles2quat(90, 90, 0),
    angles2quat(90, 90, 90),
], grid=[0, 1, 2, 3, 6], closed=True)

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations({
    'Piecewise Slerp': s.evaluate(times),
}, figsize=(3, 3))

In [ ]:
display_animation(ani, default_mode='loop')